In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

#1. Import Data and Analysis

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/ML/Lab 01/train.csv")
valid_df = pd.read_csv("/content/drive/MyDrive/ML/Lab 01/valid.csv")
test_df = pd.read_csv("/content/drive/MyDrive/ML/Lab 01/test.csv")

In [ ]:
LABELS = ["label_1","label_2","label_3","label_4"]
train_X_dict = {}
train_y_dict = {}
valid_X_dict = {}
valid_y_dict = {}
FEATURES = train_df.columns[:-4]

In [ ]:
# nearly 70% is value 6
train_df['label_4'].value_counts()/train_df.shape[0]

6     0.699088
2     0.050806
0     0.033485
12    0.033450
7     0.032889
13    0.016900
1     0.016865
11    0.016830
10    0.016830
3     0.016795
5     0.016760
9     0.016550
4     0.016445
8     0.016304
Name: label_4, dtype: float64

In [ ]:
test_df = test_df.drop(columns=LABELS)
test_df

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_247,feature_248,feature_249,feature_250,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256
0,-1.141206,-0.279703,1.019681,2.605902,0.460391,-1.167380,-1.149165,-1.205164,2.468966,-2.729526,...,1.670388,-0.355608,-0.342747,-0.102831,-0.868282,-0.429783,0.438561,-1.625122,0.599944,-0.615575
1,-1.796767,-0.056758,0.771219,4.414086,-1.229059,1.732098,0.063241,-0.126311,1.322355,-3.461282,...,1.136570,0.826741,-0.371280,-1.492272,0.227806,0.209200,2.200402,-1.701029,1.163999,-1.570136
2,-1.388968,-0.418296,0.186543,2.803852,-1.499095,1.395180,0.739648,0.274060,1.228789,-3.081147,...,0.343188,-0.243809,0.462302,-1.511495,-0.692679,0.449235,1.377531,-1.854203,-0.040538,-2.164104
3,1.276844,-0.112323,-0.310796,2.251175,0.283670,-0.581020,-0.024439,-0.306097,2.424685,-1.714863,...,3.057573,-0.632902,1.511204,-0.658826,-0.690499,-1.077949,0.930102,-2.758352,-0.246738,-1.389013
4,-1.560815,-0.168494,1.249770,1.869219,-1.646049,0.132156,-0.664602,0.483327,1.728020,-2.258071,...,1.217447,-0.374313,1.280207,-1.546299,-0.510597,1.213396,2.497490,-1.597984,-0.163483,-1.735939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,-0.520450,-1.231046,0.823668,-0.786737,-0.544497,-1.154617,0.334333,1.775682,1.523353,-1.914443,...,0.604934,-1.514841,1.343096,-1.258448,-0.600713,-0.430942,0.035674,-2.600120,0.669488,-2.029166
746,-0.512158,-2.199031,2.003965,-1.231407,-0.056881,-0.851074,0.856941,1.686686,1.430602,-1.316440,...,0.399647,-1.177832,-0.248752,0.759927,-0.219005,1.678616,1.361874,-3.596900,2.411123,-0.136058
747,0.160726,-1.636096,-1.372266,1.134842,0.232438,0.352374,-1.142442,0.364165,1.111855,-2.124050,...,-0.017546,-0.098299,0.910971,-2.589724,2.165812,-2.229485,2.895952,-0.350322,-0.400452,-1.012769
748,0.546223,-0.257980,2.767929,0.483799,1.346494,-0.963523,-0.712168,1.606215,0.785446,-0.188702,...,-0.372218,0.018880,-0.752377,-1.266369,0.468936,-1.298011,1.695302,-0.436444,0.212078,-2.275145


In [ ]:
train_nan = train_df.isna().sum()
print("Train\n",train_nan[train_nan>0])
valid_nan = valid_df.isna().sum()
print("Valid\n",valid_nan[valid_nan>0])
test_nan = test_df.isna().sum()
print("Test\n",test_nan[test_nan>0])

Train
 label_2    480
dtype: int64
Valid
 label_2    14
dtype: int64
Test
 Series([], dtype: int64)


In [ ]:
all_feature_scalers = {}
for label in LABELS:
    scaler = StandardScaler()
    train_temp = train_df.dropna(subset=[label])
    train_X_dict[label] = pd.DataFrame(scaler.fit_transform(train_temp.drop(LABELS,axis=1)),columns=FEATURES)
    train_y_dict[label] = train_temp[label]

    valid_temp = valid_df.dropna(subset=[label])
    valid_X_dict[label] = pd.DataFrame(scaler.transform(valid_temp.drop(LABELS,axis=1)),columns=FEATURES)
    valid_y_dict[label] = valid_temp[label]

    all_feature_scalers[label] = scaler


# 2. train form all features

In [ ]:
def print_accurcy_and_get_model(X,y,X_valid,y_valid):
  model=SVC(kernel='linear',class_weight='balanced')
  model.fit(X,y)
  y_pred = model.predict(X_valid)
  print(accuracy_score(y_pred,y_valid))
  return model

all_feature_models = {}

In [ ]:
for label in LABELS:
  all_feature_models[label] = print_accurcy_and_get_model(train_X_dict[label],train_y_dict[label],valid_X_dict[label],valid_y_dict[label])

0.9906666666666667
0.8804347826086957
0.9986666666666667
0.9373333333333334


# 3. Feature Removal



In [ ]:
correlation_matrix =train_X_dict[LABELS[0]].corr()

# Set the correlation threshold
threshold = 0.8

# Find columns with correlations higher than the threshold
high_correlation_columns = []
for col in correlation_matrix.columns:
    correlated_cols = correlation_matrix[correlation_matrix[col] > threshold].index.tolist()
    correlated_cols.remove(col)  # Remove self-correlation
    if correlated_cols:
        high_correlation_columns.append(col)

print("Columns with correlations higher than 0.8:")
print(high_correlation_columns)

Columns with correlations higher than 0.8:
[]


In [ ]:
from sklearn.feature_selection import VarianceThreshold
v_threshold = VarianceThreshold(threshold=0)
v_threshold.fit(train_X_dict[LABELS[0]]) # fit finds the features with zero variance
v_threshold.get_support( )

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

# 4. Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

selectKBest_models = {}
selectKBest_X = {}
selectKBest_selecters = {}
for label in LABELS:
  selecter = SelectKBest(f_classif, k=100)
  X_train_new = selecter.fit_transform(train_X_dict[label], train_y_dict[label])
  X_valid_new = selecter.transform(valid_X_dict[label])
  selectKBest_models[label] = print_accurcy_and_get_model(X_train_new,train_y_dict[label],X_valid_new,valid_y_dict[label])
  print("Diamentions ",X_train_new.shape)
  selectKBest_X[label] = X_train_new
  selectKBest_selecters[label] = selecter
# a = SelectKBest(f_classif, k=100).fit_transform(train_X_dict[label], train_y_dict[label])

0.976
Diamentions  (28520, 100)
0.7866847826086957
Diamentions  (28040, 100)
0.9973333333333333
Diamentions  (28520, 100)
0.86
Diamentions  (28520, 100)


In [ ]:
from sklearn.metrics.cluster import pair_confusion_matrix
from sklearn.decomposition import PCA

pca_models = {}
pca_X = {}
pcas = {}
for label in LABELS:
  pca = PCA(n_components=0.99,svd_solver='full')
  X_train_new = pca.fit_transform(train_X_dict[label], train_y_dict[label])
  X_valid_new = pca.transform(valid_X_dict[label])
  pca_models[label] = print_accurcy_and_get_model(X_train_new,train_y_dict[label],X_valid_new,valid_y_dict[label])
  print("Diamentions ",X_train_new.shape)
  pca_X[label] = X_train_new
  pcas[label] = pca

0.9893333333333333
Diamentions  (28520, 106)
0.8233695652173914
Diamentions  (28040, 106)
0.9986666666666667
Diamentions  (28520, 106)
0.8666666666666667
Diamentions  (28520, 106)


# 5. Export CSV

best prediction is PCA prediction

In [ ]:
all_pred = {}
best_pred = {}
new_features = {}
for label in LABELS:
  all_pred[label] = all_feature_models[label].predict(all_feature_scalers[label].transform(test_df))
  new = pcas[label].transform(test_df)
  new_features[label] = new
  best_pred[label] = pca_models[label].predict(new)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [ ]:
initial_labels = ["Predicted labels before feature engineering", "Predicted labels after feature engineering","No of new features"]
new_feature_labels = [f"new_feature_{i}" for i in range(1,257)]
new_columns = initial_labels + new_feature_labels

In [ ]:
for label in LABELS:
  data = {column_name: array for column_name, array in zip(new_columns, [all_pred[label], best_pred[label],new_features[label].shape[1]] + [new_features[label][:, i] for i in range(106)])}
  df = pd.DataFrame(data)
  df.to_csv(f"/content/drive/MyDrive/ML/Lab 01/submit/190453A_{label}.csv",index=False)
